Grab relevant metrics from github.

In [40]:
!pip install -q --pre github3.py

In [10]:
import json
import os

with open("kernels/kernels.json") as f:
    kernels = json.load(f)

In [11]:
import github3 as gh3
gh = gh3.login(os.environ["GITHUB_USER"], token=os.environ["GITHUB_TOKEN"])

In [34]:
repos = []
for key, kernel in kernels["kernels"].items():
    try:
        owner, name = kernel["actions"]["act:github"].split(":")[1].split("/")
    except:
        continue
    repos.append([kernel, gh.repository(owner, name)])

Update metrics

In [38]:
for kernel, repo in repos:
    kernel.setdefault("metrics", {}).update({
        "gh:stargazers":  repo.stargazers_count,
        "gh:forks":  repo.fork_count,
        "gh:updated": repo.updated_at.isoformat()
    })

Find logo

In [53]:
for kernel, repo in repos:
    for image in gh.search_code("repo:{} extension:png".format(repo.full_name)):
        if image.name == "logo-64x64.png":
            kernel["logo"] = (image.html_url
                .replace("github.com", "cdn.rawgit.com")
                .replace("/blob/", "/")
            )

In [57]:
for kernel, repo in repos:
    if "description" not in kernel:
        kernel["description"] = repo.description

In [58]:
with open("kernels/kernels.json", "w") as f:
    json.dump(kernels, f, indent=2, sort_keys=True)

In [17]:
scheme = repos["kern:calysto-scheme"]

In [40]:
result = list(gh.search_code("repo:{} extension:png".format(scheme.full_name)))

In [45]:
x = result[0]

In [49]:
x.html_url

'https://github.com/Calysto/calysto_scheme/blob/917d855dcae3ebc1f6f2c2024a48c78570421f30/calysto_scheme/images/logo-32x32.png'

In [55]:
repo.description

'A Haskell kernel for IPython.'